In [ ]:
import cv2
import math

import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.hub import load_state_dict_from_url
import sklearn
import pandas as pd

from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split
from torchvision.datasets import VisionDataset
import os
from torch import optim

# A simple MLP MODEL with 2 FCL

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPModel, self).__init__()

        # Define layers
        self.Flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        self.loss_fn = nn.CrossEntropyLoss()
        #Adam
        #self.optimizer = optim.Adam(self.parameters(), lr=0.001) #weight decay
        self.optimizer = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        self.running_loss = 0
        self.loss = None
        self.losses = []

    def forward(self, x, target =None):
        # Forward pass
        x = self.Flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x
    def fit(self, x, targets):
        #train/optimize/fit
        preds = self.forward(x)
        self.loss = self.loss_fn(preds, targets)
        self.loss.backward()
        self.optimizer.step()
        
        loss_item = self.loss.item()
        return loss_item


    def reset_loss(self, value):
        self.running_loss = value
        self.losses = []
        return

# CNN MODEL 3 CONV & 3 FCL

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_size,input_channel, kernel_size, middle_channel, output_channel,hidden_input,output_size,padding = 0, Drop = 0.1):
        super(CNNModel, self).__init__()
        reduce_size = 0
        size = input_size
        if padding == 0:
            reduce_size = size - kernel_size +1
            reduce_size = reduce_size/2
            reduce_size = reduce_size - kernel_size +1
            reduce_size = int(reduce_size/2)
        else:
            reduce_size = int(size/4)
        # Define layers
        self.Drop = Drop
        self.Dropout = nn.Dropout(p=self.Drop)
        self.Conv2d1 = nn.Conv2d(in_channels=input_channel,out_channels=middle_channel, kernel_size=kernel_size,padding = padding)
        self.ReLU = nn.ReLU()
        self.MaxPool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Conv2d2 = nn.Conv2d(in_channels=middle_channel,out_channels=output_channel, kernel_size=kernel_size,padding = padding)
        self.Flatten = nn.Flatten()
        self.fc1 = nn.Linear(output_channel*reduce_size*reduce_size,hidden_input)
        self.fc2 = nn.Linear(hidden_input,output_size)
        
        self.loss_fn = nn.CrossEntropyLoss()
        #self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.optimizer = optim.SGD(self.parameters(), lr=0.001,momentum=0.9) 
        self.running_loss = 0
        self.loss = None
        self.losses = []
        self.dropout_enable = False
        
    def forward(self, x, target = None):
        #x shape: (batch_size, h, w, channels)
        #output shape: (batch_size, output_size)
        # Forward pass
        #if self.Drop:
        #    x = self.Dropout(x)
            
        x = self.Conv2d1(x)
        x = self.ReLU(x)
        
        if self.dropout_enable:
            x = self.Dropout(x)
            
        x = self.MaxPool2d(x)
        x = self.Conv2d2(x)
        x = self.ReLU(x)
        x = self.MaxPool2d(x)
        x = self.Flatten(x)
        x = self.fc1(x)
        x = self.ReLU(x)
        if self.dropout_enable:
            x = self.Dropout(x)
        x = self.fc2(x)

        #update
        
        return x


    def fit(self, x, targets):
        #train/optimize/fit
        preds = self.forward(x)
        self.loss = self.loss_fn(preds, targets)
        self.loss.backward()
        self.optimizer.step()
        
        loss_item = self.loss.item()
        return loss_item


    def reset_loss(self):
        self.running_loss = 0
        self.loss = None
        return